In [2]:
import numpy as np
import scipy
import soundfile as sf
import sklearn
import scipy.fftpack
import os

In [3]:
l=os.listdir("/Sivasankar/UCSD/Winter2019/CSE237A/finalproj/audio/test")

In [4]:
v=len(l)

In [5]:
s=np.zeros((v,480000))
f=np.zeros(v)
for i in range(0,v):
    s[i],f[i]=sf.read(l[i])

In [6]:
overlap=int(f[1]*0.01)
no_frames=int(len(s[1])/overlap)
framesize=int(f[1]*0.025)
no_frames,framesize,overlap

(1000, 1200, 480)

In [7]:
frames=np.zeros((v,no_frames,framesize))
for k in range(0,v):
    for i in range(0,no_frames):
        for j in range(0,framesize):
            t=i*overlap+j
            if t<len(s[k]):
                frames[k][i][j]=s[k][t]
            

In [8]:
def fftfunc(fr,nfft):
    l=np.fft.rfft(fr,nfft)
    l=np.absolute(l)
    l=(1/2048)*(np.square(l))
    print(l)
    return l

In [9]:
comp_spec=np.ndarray((v,no_frames,1025))
for i in range(0,v):
    comp_spec[i]=fftfunc(frames[i],2048)

C:\Users\psiva\Anaconda3\lib\site-packages\mkl_fft\_numpy_fft.py:331: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.rfft_numpy(a, n=n, axis=axis)


[[8.30361689e-03 9.83832782e-02 4.86686080e-01 ... 4.45886071e-07
  1.00808445e-06 9.91753950e-08]
 [3.04914817e-02 3.30475185e-01 3.12937873e-01 ... 4.48813024e-07
  5.00636829e-07 3.90777586e-07]
 [1.70055267e-01 2.24382747e-01 1.24524571e-01 ... 5.87214471e-08
  2.88812482e-08 4.28594831e-08]
 ...
 [1.25865522e-01 3.45059652e-02 1.82674096e-01 ... 1.78903825e-06
  3.16214682e-06 1.07009146e-06]
 [1.64927664e-01 6.46367148e-03 2.85509397e-01 ... 5.02504304e-07
  1.32327183e-06 6.88128239e-08]
 [6.85738673e-02 6.69540443e-02 7.43520566e-02 ... 1.03854514e-06
  6.28149555e-07 3.50557457e-07]]
[[2.08937865e-02 1.78979673e-02 3.06615718e-03 ... 2.38042484e-08
  1.35653149e-08 6.36082405e-08]
 [6.14793250e-02 2.14387897e-02 8.82891980e-05 ... 6.06087910e-09
  2.98390358e-09 8.91304808e-09]
 [3.37918775e-02 1.77508752e-02 1.18135785e-03 ... 4.22922866e-09
  3.06622439e-08 3.93310984e-09]
 ...
 [5.99447162e-04 6.38680819e-04 1.62749244e-04 ... 2.05366650e-06
  2.46805661e-06 3.01057707e-06]

In [10]:
lowfreq=0
highfreq=2595*np.log10(1+24000/700.)

melpoints = np.linspace(lowfreq,highfreq,22)

melpoints1=700*(10**(melpoints/2595.0)-1)

bins=np.floor((2049)*melpoints1/48000)

fbank = np.zeros((20,1025))
for j in range(0,20):
    for i in range(int(bins[j]), int(bins[j+1])):
        fbank[j,i] = (i - bins[j]) / (bins[j+1]-bins[j])
    for i in range(int(bins[j+1]), int(bins[j+2])):
        fbank[j,i] = (bins[j+2]-i) / (bins[j+2]-bins[j+1])

In [11]:
feat1=np.ndarray((v,1000,20))
feat=np.ndarray((v,1000,13))
for i in range(0,v):
    feat1[i]=np.dot(comp_spec[i],fbank.T)
    feat1[i]=np.where(feat1[i]==0,np.finfo(float).eps,feat1[i])
    feat1[i]=np.log(feat1[i])
    feat1[i]=scipy.fftpack.dct(feat1[i],norm='ortho')
    feat[i]=feat1[i][:,:13]

In [12]:
pad=np.zeros((v,1000,17))
delta=np.zeros((v,1000,13))
dpad=np.zeros((v,1000,17))
ddelta=np.zeros((v,1000,13))
for k in range(0,v):
    for i in range(0,1000):
        for j in range(2,15):
            pad[k][i][j]=feat[k][i][j-2]
for k in range(0,v):
    for i in range(0,1000):
        for j in range(0,13):
            delta[k][i][j]=np.dot(np.arange(-2,3),pad[k][i][j:j+5])/10
for k in range(0,v):
    for i in range(0,1000):
        for j in range(2,15):
            dpad[k][i][j]=pad[k][i][j-2]
for k in range(0,v):
    for i in range(0,1000):
        for j in range(0,13):
            ddelta[k][i][j]=np.dot(np.arange(-2,3),dpad[k][i][j:j+5])/10  

In [13]:
featvect=np.zeros((v,1000,39))
for k in range(0,v):
    for i in range(0,1000):
        for j in range(0,13):
            featvect[k][i][j]=feat[k][i][j]
            featvect[k][i][j+13]=delta[k][i][j]
            featvect[k][i][j+26]=ddelta[k][i][j]
featvect=featvect.reshape(v,1,39000)

for k in range(0,v):
    np.savetxt('test%s.txt'%str(k+1),featvect[k])

In [14]:
featvect[0].shape

(1, 39000)